In [ ]:
import numpy as np
import pandas as pd
from parflow.tools.io import read_pfb, write_pfb
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
pf_output_dir = 'path/to/parflow/run/outputs/'
pf_input_dir = 'path/to/parflow/run/inputs/'
forcing_dir = 'path/to/forcing/files/'
runname = '<parflow_runname>'

### Check a file for negative pressures below a given threshold

In [ ]:

file = f'{pf_output_dir}/{runname}.out.press.04699.pfb'
print(file)

press = read_pfb(file)
press[press < -1.0e+38] = np.nan
print(np.shape(press))
print(np.nanmax(press))
print(np.nanmin(press))

nthreshold = -50.
indices_press_neg = np.argwhere(press[:,:,:] < nthreshold)

print(indices_press_neg)
print(len(indices_press_neg))

### Check a file for positive pressures above a given threshold

In [ ]:
layer = 9

print(np.nanmax(press[layer,:,:]))
print(np.nanmin(press[layer,:,:]))
threshold = 50

indices_press = np.argwhere(press[layer,:,:] > threshold)

print(indices_press)
print(len(indices_press))

### Plot outside-threshold positive and negative pressures on a map of the pressure field

In [ ]:
plt.figure()
plt.imshow(press[layer,:,:], origin='lower', cmap='plasma',norm=colors.LogNorm(vmin = 0.00001, vmax = 50))
cbar = plt.colorbar(orientation='horizontal',shrink=0.15)
plt.scatter(indices_press[:,1],indices_press[:,0],facecolors='none', edgecolors='blue', s=3,marker='o',alpha=0.5,linewidths=0.5)
plt.scatter(indices_press_neg[:,2],indices_press_neg[:,1],facecolors='none', edgecolors='red', s=3,marker='o',alpha=0.5,linewidths=0.5)
plt.savefig('surf_press.png', dpi=2200,bbox_inches='tight')

### Check the min and max of a saturation file

In [ ]:
sat_file = f'{pf_output_dir}/{runname}.out.satur.01315.pfb'

print(sat_file)
sat = read_pfb(sat_file)
sat[sat < -1.0e+38] = np.nan
print(np.shape(sat))
print(np.nanmax(sat))
print(np.nanmin(sat))

### Plot below-threshold negative pressure locations overlaid on saturation map

In [ ]:
plt.figure()
plt.imshow(sat[9,:,:], origin='lower', cmap='plasma')
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.scatter(indices_press_neg[:,2],indices_press_neg[:,1],facecolors='none', edgecolors='red', s=3,marker='o',alpha=0.5,linewidths=0.5)

plt.savefig('surf_sat.png', dpi=1000,bbox_inches='tight')

### Check the min and max values of the mask file and plot

In [ ]:
mask_file = f'{pf_output_dir}/{runname}.out.mask.pfb'

print(mask_file)
mask = read_pfb(mask_file)
print(np.shape(mask))
print(np.nanmax(mask))
print(np.nanmin(mask))

plt.figure()
plt.imshow(mask[0,:,:], origin='lower', cmap='plasma')
plt.axis('off')
plt.savefig('mask.png', dpi=500,bbox_inches='tight')

### Check the min and max values of the top patch file and plot

In [ ]:
top_patch_file = f'{pf_output_dir}/{runname}.out.top_patch.pfb'

print(top_patch_file)
top = read_pfb(top_patch_file)
top[top <= 0] = np.nan
print(np.shape(top))
print(np.nanmax(top))
print(np.nanmin(top))

plt.figure()
plt.imshow(top[0,:,:], origin='lower', cmap='viridis')
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('top.png', dpi=1500,bbox_inches='tight')

### Get the indices from the top patch file of sinks and lakes

In [ ]:
threshold = 3
indices_sink = np.argwhere(top[0,:,:] > threshold)

print(indices_sink)
print(len(indices_sink))

threshold = 3
indices_lake = np.argwhere(top[0,:,:] == threshold)

print(indices_lake)
print(len(indices_lake))

### Check sink locations against locations with above-threshold positive pressure

In [ ]:
def find_matching_pairs(list1, list2):
    """Finds matching pairs of indices between two lists of paired indices.

    Args:
        list1 (list): A list of paired indices (e.g., [(1, 2), (3, 4)]).
        list2 (list): Another list of paired indices.

    Returns:
        list: A list of the matching pairs.
    """

    matching_pairs = []
    for pair1 in list1:
        if pair1 in list2:
            matching_pairs.append(pair1)

    return matching_pairs

In [ ]:
matching_indices = find_matching_pairs(indices_press, indices_sink)
print(matching_indices)
print(len(matching_indices))

### Check the min and max values of a CLM output file

In [ ]:
clm_file = f'{pf_output_dir}/{runname}.out.clm_output.04030.C.pfb'

print(clm_file)
CLM = read_pfb(clm_file)
CLM[CLM == -9999.0] = np.nan
print(np.shape(CLM))
print(np.nanmax(CLM))
print(np.nanmin(CLM))

### Check the min and max values of SWE in a single CLM output file and plot

In [ ]:
#CLM data array format 
# reading the CLM file PFCLM_SC.out.clm_output.<file number>.C.pfb
# variables are by layer:
# 0 eflx_lh_tot:  total latent heat flux (Wm-2)  
# 1 eflx_lwrad_out: total upward LW radiation (Wm-2)  
# 2 eflx_sh_tot: total sensible heat flux (Wm-2)  
# 3 eflx_soil_grnd: ground heat flux (Wm-2)    
# 4 qflx_evap_tot: net veg. evaporation and transpiration and soil evaporation (mms-1)  
# 5 qflx_evap_grnd: ground evaporation (mms-1)   
# 6 qflx_evap_soi: soil evaporation (mms-1)   
# 7 qflx_evap_veg: vegetation evaporation (canopy) and transpiration (mms-1)   
# 8 qflx_tran_veg: transpiration (mms-1)  
# 9 qflx_infl: infiltration flux (mms-1)   
# 10 swe_out: SWE (mm)   
# 11 t_grnd: ground temperature (K)  
# 12 irrigation flux
# 13 - 24 Soil temperature by layer (K)
print(np.nanmax(CLM[10,:,:]))
print(np.nanmin(CLM[10,:,:]))
plt.figure()
plt.imshow(CLM[10,:,:], origin='lower', cmap='ocean_r',vmin = 1, vmax = 3000)
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.2) 
plt.savefig('clm.png', dpi=1200,bbox_inches='tight')

### Check the min and max of a precipitation forcing file and plot a specific timestep

In [ ]:
precip_file = f'{forcing_dir}/CW3E.APCP.000001_to_000024.pfb'

print(precip_file)
APCP = read_pfb(precip_file)
APCP[APCP < -1.0e+38] = np.nan
print(np.shape(APCP))
print(np.nanmax(APCP))
print(np.nanmin(APCP))


time = 0
plt.figure()
plt.imshow(APCP[time,:,:]*3600, origin='lower', cmap='plasma')
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('precip.png', dpi=1200,bbox_inches='tight')
print(np.nanmax(APCP[time,:,:]*3600))
print(np.nanmin(APCP[time,:,:]*3600))
print(np.argmax(APCP[time,:,:]))

### Check min and max of slope x values and plot

In [ ]:
slopex_file = f'{pf_output_dir}/{runname}.out.slope_x.pfb'

print(slopex_file)
Sx = read_pfb(slopex_file)
print(np.shape(Sx))
print(np.nanmax(Sx))
print(np.nanmin(Sx))

plt.figure()
plt.imshow(Sx[0,:,:], origin='lower', cmap='bwr',vmin = -.5, vmax = 0.5)
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('Sx.png', dpi=1200,bbox_inches='tight')
print(np.nanmax(Sx[0,:,:]))
print(np.nanmin(Sx[0,:,:]))
print(np.argmax(Sx[0,:,:]))

### Check min and max of slope y values and plot

In [ ]:
slopey_file = f'{pf_output_dir}/{runname}.out.slope_y.pfb'

print(slopey_file)
Sy = read_pfb(slopey_file)
print(np.shape(Sy))
print(np.nanmax(Sy))
print(np.nanmin(Sy))

plt.figure()
plt.imshow(Sy[0,:,:], origin='lower', cmap='bwr',vmin = -.5, vmax = 0.5)
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('Sy.png', dpi=1200,bbox_inches='tight')
print(np.nanmax(Sy[0,:,:]))
print(np.nanmin(Sy[0,:,:]))
print(np.argmax(Sy[0,:,:]))

### Check slopes at a sink location

In [ ]:
iloc = 1

print(indices_sink[iloc])
print(Sx[0,indices_sink[iloc,0],indices_sink[iloc,1]])
print(Sy[0,indices_sink[iloc,0],indices_sink[iloc,1]])

### Check pressure at a sink location

In [ ]:
iloc = 50

print(indices_sink[iloc])
print(press[9,indices_sink[iloc,0],indices_sink[iloc,1]])

### Check pressure at a lake location

In [ ]:
iloc = 12

print(indices_lake[iloc])
print(press[9,indices_lake[iloc,0],indices_lake[iloc,1]])

### Create a list of pressure values at all of the sinks

In [ ]:
imax = len(indices_sink)
print(imax)
sink_press = press[9,indices_sink[0:imax,0],indices_sink[0:imax,1]]

In [ ]:
positive_loc = np.argwhere(sink_press > 0.0)
print(sink_press[positive_loc])
print(np.max(sink_press[positive_loc]))

### Create a list of pressure values at all of the lakes

In [ ]:
imax = len(indices_lake)
print(imax)
lake_press = press[9,indices_lake[0:imax,0],indices_lake[0:imax,1]]

In [ ]:
positive_loc = np.argwhere(lake_press > 0.0)
print(lake_press[positive_loc])
print(np.max(lake_press[positive_loc]))

### Check min and max mannings values and plot

In [ ]:
mannings_file = f'{pf_output_dir}/{runname}.out.mannings.pfb'

print(mannings_file)
mannings = read_pfb(mannings_file)
mannings[mannings == 1.0] = np.nan
print(np.shape(mannings))
print(np.nanmax(mannings))
print(np.nanmin(mannings))

plt.figure()
plt.imshow(mannings[0,:,:], origin='lower', cmap='plasma') #,vmin = -.5, vmax = 0.5)
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('mannings.png', dpi=1200,bbox_inches='tight')
print(np.nanmax(mannings[0,:,:]))
print(np.nanmin(mannings[0,:,:]))
print(np.argmax(mannings[0,:,:]))

### Inspect data within a window of a high-pressure location

In [ ]:
iloc = 3
print(mannings[0,indices_press[iloc,0],indices_press[iloc,1]])

pm = 3
col_headers = np.arange(indices_press[iloc,1]-pm, indices_press[iloc,1]+pm)
row_headers = np.arange(indices_press[iloc,0]-pm,indices_press[iloc,0]+pm)

subset_Sx = Sx[0,indices_press[iloc,0]-pm:indices_press[iloc,0]+pm,indices_press[iloc,1]-pm:indices_press[iloc,1]+pm]
subset_Sy = Sy[0,indices_press[iloc,0]-pm:indices_press[iloc,0]+pm,indices_press[iloc,1]-pm:indices_press[iloc,1]+pm]
subset_press = press[9,indices_press[iloc,0]-pm:indices_press[iloc,0]+pm,indices_press[iloc,1]-pm:indices_press[iloc,1]+pm]
subset_mann = mannings[0,indices_press[iloc,0]-pm:indices_press[iloc,0]+pm,indices_press[iloc,1]-pm:indices_press[iloc,1]+pm]
subset_mask = mask[0,indices_press[iloc,0]-pm:indices_press[iloc,0]+pm,indices_press[iloc,1]-pm:indices_press[iloc,1]+pm]
subset_top = top[0,indices_press[iloc,0]-pm:indices_press[iloc,0]+pm,indices_press[iloc,1]-pm:indices_press[iloc,1]+pm]

print(indices_press[iloc])
df_Sx = pd.DataFrame(subset_Sx, index=row_headers, columns=col_headers)
print("Sx")
print(df_Sx.to_string())
df_Sy = pd.DataFrame(subset_Sy, index=row_headers, columns=col_headers)
print("Sy")
print(df_Sy.to_string())
df_mann = pd.DataFrame(subset_mann, index=row_headers, columns=col_headers)
print("Mannings")
print(df_mann.to_string())
df_press = pd.DataFrame(subset_press, index=row_headers, columns=col_headers)
print("Press")
print(df_press.to_string())
print("Mask")
df_mask = pd.DataFrame(subset_mask, index=row_headers, columns=col_headers)
print(df_mask.to_string())
print("Top")
df_top = pd.DataFrame(subset_top, index=row_headers, columns=col_headers)
print(df_top.to_string())

# not plotting origin=lower since the Y axis on the matrices is all backwards
plt.figure()
plt.imshow(subset_press, cmap='plasma')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.show()

### Check min and max values of input mannings file and plot

In [ ]:
input_mannings_file = f'{pf_input_dir}/CONUS2.0.Final1km_mannings_rv50_original_values_Lake_Sink.pfb'

print(input_mannings_file)
mannings = read_pfb(input_mannings_file)
mannings[mannings == 1.0] = np.nan
print(np.shape(mannings))
print(np.nanmax(mannings))
print(np.nanmin(mannings))

plt.figure()
plt.imshow(mannings[0,:,:], origin='lower', cmap='plasma',vmin = 5.6e-6, vmax = 1.e-4)
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('mannings.png', dpi=1200,bbox_inches='tight')
print(np.nanmax(mannings[0,:,:]))
print(np.nanmin(mannings[0,:,:]))
print(np.argmax(mannings[0,:,:]))

### Check min and max values of porosity and plot

In [ ]:
porosity_file = f'{pf_output_dir}/{runname}.out.porosity.pfb'

print(porosity_file)
poro = read_pfb(porosity_file)
poro[poro == 1.0] = np.nan
print(np.shape(poro))
print(np.nanmax(poro))
print(np.nanmin(poro))


plt.figure()
plt.imshow(poro[2,:,:], origin='lower', cmap='plasma')
plt.axis('off')
cbar = plt.colorbar(orientation='horizontal',shrink=0.15) 
plt.savefig('poro.png', dpi=1500,bbox_inches='tight')

### Example: Correct mannings values at high-pressure location

In [ ]:
# mannings fix for high pressure off the MISS river
# load last mannings file
file = "CONUS2.0.Mannings_906.pfb"

print(file)
mannings_old = read_pfb(file)
print(np.shape(mannings_old))
print(np.nanmax(mannings_old))
print(np.nanmin(mannings_old))

# confirm that mannings stream value is too high
print("high point:",mannings_old[0,775,2734])

# check adjacent value
print("stream value:",mannings_old[0,774,2734])

# set high point to correct value
mannings_old[0,775,2734] = 8.33e-6

# confirm that mannings stream value is fixed
print("old high point:",mannings_old[0,775,2734])

# save as PFB with correct distribution 
write_pfb("CONUS2.0.Mannings_906v1.pfb", mannings_old, p=70, q=48, r=1, x=0.0, y=0.0, z=0.0, dx=1000.0, dy=1000.0, dz=1.0, dist=True)

### Example: Correct slope values at high-pressure location

In [ ]:
# fix slopes for second high point
# change the Y slope of [1464 3977] from zero to 0.0001
file = 'model_inputs/CONUS2.0.Final1km.slopey.pfb'
print(file)
Sy_old = read_pfb(file)
print(np.shape(Sy_old))
print(np.nanmax(Sy_old))
print(np.nanmin(Sy_old))

# confirm that slopey stream value is too high
print("zero value at coast:",Sy_old[0,1464,3977])

# set zero point to correct value
Sy_old[0,1464,3977] = 0.0001

# confirm that slopey stream value is fixed
print("old zero point, now fixed:",Sy_old[0,1464,3977])

# save as PFB with correct distribution 
write_pfb("CONUS2.0.Final1km.slopeyv1.pfb", Sy_old, p=70, q=48, r=1, x=0.0, y=0.0, z=0.0, dx=1000.0, dy=1000.0, dz=1.0, dist=True)